Get Hourly SPY Data Since 1/1/22

In [ ]:
import yfinance as yf
import pandas as pd
import seaborn as sns
import matplotlib as plt
from sklearn.model_selection import TimeSeriesSplit

from datetime import datetime
data = yf.download('SPY','2022-01-01','2023-09-30', interval ="60m")
import matplotlib.pyplot as plt
%matplotlib inline

data['Adj Close'].plot()
plt.ion()
print(data)

Now we need to split the data.


In [ ]:
#data.reset_index(inplace=True)


# maybe consider splitting by whole days here
# 20% test split but could do more
# or 30% or 50%
train_size = 2400

data_train = data[:train_size]
data_test = data[train_size:]

test_size = len(data_test)
# 656 as of today

data_test = data_test[["Datetime", "Close"]]
data_train = data_train[["Datetime", "Close"]]

seriesTrain = data_train.set_index('Datetime')
seriesTest = data_test.set_index('Datetime')





'''
clean_test = data_test

clean_test['unique_id'] = range(0, len(clean_test['Close']))

clean_test.rename({"Datetime" : "ds", "Close": "y"}, axis = "columns", inplace = True)
clean_test = pd.DataFrame(clean_test)

clean_test = clean_test[['unique_id', 'ds', 'y']]
'''

Attempting to implement with sklearn tscv 

In [ ]:
tscvData = data[["Close"]]


# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

tscv = TimeSeriesSplit(gap=0 max_train_size=None, n_splits=5, test_size=1)

for i, (train_index, test_index) in enumerate(tscv.split(tscvData)):
    print(i, "train: ", train_index, "test: ", test_index)
    model = auto_arima(tscvData[train_index])
    predictions = model.predict(n_periods = 1)
    print(predictions)

In [ ]:
def crossValidation(train, test, window, predReturnFunction):

    totalPredictions = pd.Series([])
    windows = len(test) // window

    for i in range(0, windows):
        print("Window: ", i)

        addNum = i * window
        intermediateData = train.append(test[:addNum])
        #print(test[:addNum])
        prediction = predReturnFunction(intermediateData, window)
        
        print(prediction)
        #print(prediction.shape)

        #for i in range(1, len(prediction)):
        #    totalPredictions = totalPredictions.append#(prediction)
        Series.append(totalPredictions, prediction, verify_integrity = True)
        print(totalPredictions)


    #remove hanging dates if somem were not included in a window
    # to make test indices the same length as the predictions
    #testSheared = test[:(windows*window)]
    #totalPredictions.index = testSheared.index

    return totalPredictions


And then we need to train the statistical models


Import packages

In [ ]:
from pmdarima.arima import auto_arima
from pmdarima import model_selection

Run Prediction

In [ ]:
def aaPredFunction(dataIn, windowSize):
    model = auto_arima(dataIn, seasonal = False, stepwise = False)
    predictions = model.predict(n_periods = windowSize)
    return predictions

#aaModel = auto_arima(seriesTrain)
test_out_size = 3


tempTest = seriesTest[:test_out_size]
#print(model_selection.cross_val_predict(, tempTest))

tempModel = auto_arima(seriesTrain)

tempModel.summary()
#cv = model_selection.RollingForecast(window_size=1, step=1, h=1)
#model1_cv_scores = model_selection.cross_val_score(
#   tempModel, seriesTest, scoring='smape', cv=cv, verbose=2)

#model1_cv_scores
aaPrediction = crossValidation(seriesTrain, tempTest, 1, aaPredFunction)

Let's try and visualize the cross-validated auto-arima performance


In [ ]:
#print(aaPrediction[0].tostring)
#aaPrediction = pd.DataFrame(aaPrediction)
aaPrediction[1].plot()

#sns.lineplot(aaPrediction, x = aaPrediction.index, y = aaPrediction)

In [ ]:
backN = 50

smallTest = seriesTest[:test_out_size]
smallTrain = seriesTrain[-backN:]

smallTest["x"] = range(backN + 1,backN + test_out_size + 1)
smallTrain["x"] = range(1,backN + 1)

#aaPrediction = pd.Series(aaPrediction)
#aaPrediction.index = smallTest.index
#aaPrediction = pd.Series(aaPrediction)
#aaPrediction.index = smallTest.index
aaPrediction2 = pd.DataFrame([float(x) for x in aaPrediction])
aaPrediction2["x"] = range(backN + 1, backN + test_out_size + 1)
aaPrediction2.rename(columns = {0 : "val", "x": "x"}, inplace = True )
aaPrediction2 = pd.DataFrame(aaPrediction2)

print(aaPrediction)

plt.figure(figsize= (8,5) )
plt.plot(smallTrain["x"], smallTrain["Close"], label = "Training")
plt.plot(smallTest["x"], smallTest["Close"], label = "Test")
plt.plot(aaPrediction2["x"], aaPrediction2["val"], label = "Auto Arima")
plt.legend(loc = 'upper left')
plt.show()

And then we can reformat data for NN
